<a href="https://colab.research.google.com/github/priya-guruswamy/TOS-Trade-Log/blob/main/Appended_Tradelog_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# AGGREGATED TRADES ALONE
import csv
import os
import pandas as pd
from itertools import chain
from dateutil.parser import parse


# Directory to load files from
directory_path = r'C:\Users\bguru\Documents\Trading\Trade Analysis\TOS Statement Download'
output_directory = r'C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog'  # Directory for saving outputs

# List files in the directory
files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]
if not files:
    print(f"No files found in directory: {directory_path}")
    exit()

print("Available files:")
for idx, file in enumerate(files):
    print(f"{idx + 1}: {file}")

# Prompt the user to select a file
while True:
    try:
        file_choice = int(input("Enter the number corresponding to the file you want to process: "))
        if 1 <= file_choice <= len(files):
            file_path = os.path.join(directory_path, files[file_choice - 1])
            file_name = files[file_choice - 1]  # Extract the file name
            print(f"Selected file: {file_path}")
            break
        else:
            print("Invalid choice. Please select a valid file number.")
    except ValueError:
        print("Invalid input. Please enter a number.")

# Extract the date from the input file name (assuming the format is `YYYY-MM-DD-*`)
try:
    date_part = file_name.split('-')[0] + '-' + file_name.split('-')[1] + '-' + file_name.split('-')[2]
except IndexError:
    raise ValueError("The file name format does not contain a valid date.")

# Create output file paths with the date in the name
output_file_path = os.path.join(output_directory, f"{date_part}-Aggregated_Trades.csv")

# Data storage
cashBalance = []
futuresStatement = []
forexStatements = []
accountOrderHistory = []
totalCash = []
accountTradeHistory = []
profitsandLosses = []
forexAccountSummary = []
accountSummary = []

# Read CSV file
with open(file_path) as statement:
    statement = csv.reader(statement, delimiter='\n')
    count, breakCount = 0, 0

    for line in statement:
        count += 1

        if count < 5:
            continue
        if len(line) == 0:
            breakCount += 1
        if breakCount < 1:
            cashBalance.append(line)
        if breakCount == 1 and len(line) != 0:
            futuresStatement.append(line)
        if breakCount == 2 and len(line) != 0:
            forexStatements.append(line)
        if breakCount == 3 and len(line) != 0:
            totalCash.append(line)
        if breakCount == 5 and len(line) != 0:
            accountOrderHistory.append(line)
        if breakCount == 6 and len(line) != 0:
            accountTradeHistory.append(line)

# Process CSV data into a pandas DataFrame
def orderSplitter(subsection: list) -> pd.DataFrame:
    subsection.pop(0)
    for i, j in enumerate(subsection):
        j[0] = j[0].split(',')

    return pd.DataFrame(
        list(chain.from_iterable(subsection[1:])),
        columns=list(chain.from_iterable(subsection[0]))
    ).iloc[:, 1:].iloc[::-1].reset_index().drop(['index'], axis=1)

orders = orderSplitter(accountTradeHistory)

# Filter out rows where "Spread" is VERTICAL, COMBO, or blank
filtered_orders = orders[
    ~orders['Spread'].isin(['VERTICAL', 'COMBO', 'COVERED', ''])  # Filter out these spreads
].reset_index(drop=True)

# Separate stocks and options
stocks = filtered_orders[filtered_orders['Spread'] == 'STOCK']  # Stocks
options = filtered_orders[filtered_orders['Spread'] == 'SINGLE']  # Options

# Group by symbols
def groupBySymbol(filtered_df):
    grouped = {}
    for symbol in set(filtered_df['Symbol']):
        grouped[symbol] = filtered_df[filtered_df['Symbol'] == symbol]
    return grouped

stockOrdersBySymbol = groupBySymbol(stocks)
optionOrdersBySymbol = groupBySymbol(options)

# Analyze trades
def processMatchedTrades(symbol, positions, isStock=True):
    openTrades = []
    aggregatedTrades = []

    for _, row in positions.iterrows():
        qty = int(row['Qty'].replace('+', '').replace('-', ''))
        price = float(row['Price'])
        timestamp = row['Exec Time']  # Use the correct column for timestamp

        if row['Pos Effect'] == 'TO OPEN':
            openTrades.append({
                'Symbol': symbol, 'Description': 'Stock' if isStock else f"{row['Spread']} {row['Exp']} {row['Strike']} {row['Type']}",
                'Direction': 'Long', 'Time Opened': timestamp, 'Time Closed': '',
                'Holding Period (minutes)': '', 'Quantity': qty, 'Average Buy Price': price,
                'Average Sell Price': '', 'Profit/Loss per Unit': '', 'Profit/Loss': ''
            })
        elif row['Pos Effect'] == 'TO CLOSE':
            sellQty = qty
            sellPrice = price
            sellTimestamp = timestamp

            weightedSellPrice = 0
            totalMatchedQty = 0
            weightedBuyPrice = 0
            buyTimestamp = None

            while sellQty > 0 and openTrades:
                buyTrade = openTrades.pop(0)
                matchQty = min(sellQty, buyTrade['Quantity'])
                sellQty -= matchQty
                buyTrade['Quantity'] -= matchQty

                weightedSellPrice += matchQty * sellPrice
                weightedBuyPrice += matchQty * buyTrade['Average Buy Price']
                totalMatchedQty += matchQty

                if buyTrade['Quantity'] > 0:
                    openTrades.insert(0, buyTrade)

                buyTimestamp = buyTrade['Time Opened']

            if totalMatchedQty > 0:
                avgSellPrice = weightedSellPrice / totalMatchedQty
                avgBuyPrice = weightedBuyPrice / totalMatchedQty
                profitLoss = (avgSellPrice - avgBuyPrice) * totalMatchedQty
                holdingPeriod = (parse(sellTimestamp) - parse(buyTimestamp)).total_seconds() / 60.0

                aggregatedTrades.append([
                    symbol, 'Stock' if isStock else f"{row['Spread']} {row['Exp']} {row['Strike']} {row['Type']}", 'Long',
                    buyTimestamp, sellTimestamp, holdingPeriod, totalMatchedQty,
                    avgBuyPrice, avgSellPrice, profitLoss / totalMatchedQty, profitLoss
                ])

    return aggregatedTrades

# Process and aggregate trades
trades = []

# Process stock trades
for symbol, positions in stockOrdersBySymbol.items():
    stock_aggregated = processMatchedTrades(symbol, positions, isStock=True)
    trades.extend(stock_aggregated)

# Process option trades
for symbol, positions in optionOrdersBySymbol.items():
    option_aggregated = processMatchedTrades(symbol, positions, isStock=False)
    trades.extend(option_aggregated)

# Save aggregated trades to a DataFrame
aggregated_trades_df = pd.DataFrame(trades, columns=[
    'Symbol', 'Description', 'Direction', 'Time Opened', 'Time Closed',
    'Holding Period (minutes)', 'Quantity', 'Average Buy Price',
    'Average Sell Price', 'Profit/Loss per Unit', 'Profit/Loss'
])

# Specify the date-time format
date_format = '%m/%d/%y %H:%M:%S'

# Convert 'Time Opened' and 'Time Closed' to datetime format
aggregated_trades_df['Time Opened'] = pd.to_datetime(aggregated_trades_df['Time Opened'], format=date_format, errors='coerce')
aggregated_trades_df['Time Closed'] = pd.to_datetime(aggregated_trades_df['Time Closed'], format=date_format, errors='coerce')

# Ensure numeric columns are explicitly converted before grouping
numeric_columns = [
    'Holding Period (minutes)', 'Quantity', 'Average Buy Price',
    'Average Sell Price', 'Profit/Loss per Unit', 'Profit/Loss'
]

for col in numeric_columns:
    aggregated_trades_df[col] = pd.to_numeric(aggregated_trades_df[col], errors='coerce')

# Group by Symbol, Description, and Time Opened while aggregating other values
aggregated_trades_df = aggregated_trades_df.groupby(
    ['Symbol', 'Description', 'Time Opened'],
    as_index=False
).agg({
    'Direction': 'first',  # Keep the first Direction value
    'Time Closed': 'max',  # Take the latest Time Closed value
    'Holding Period (minutes)': 'sum',  # Sum Holding Period
    'Quantity': 'sum',  # Sum Quantity
    'Average Buy Price': 'mean',  # Average Buy Price
    'Average Sell Price': 'mean',  # Average Sell Price
    'Profit/Loss per Unit': 'mean',  # Average Profit/Loss per Unit
    'Profit/Loss': 'sum'  # Sum Profit/Loss
})

# Ensure 'Profit/Loss' is cast to string type before applying SINGLE-specific logic
aggregated_trades_df['Profit/Loss'] = aggregated_trades_df['Profit/Loss'].astype(float).round(2)

# Apply conversion only for SINGLE trades
single_mask = aggregated_trades_df['Description'].str.startswith('SINGLE', na=False)
aggregated_trades_df.loc[single_mask, 'Profit/Loss'] = (
    pd.to_numeric(aggregated_trades_df.loc[single_mask, 'Profit/Loss'], errors='coerce').fillna(0) * 100
).round(2)

# Convert 'Profit/Loss' back to string for saving
aggregated_trades_df['Profit/Loss'] = aggregated_trades_df['Profit/Loss'].astype(str)

# Sort the DataFrame by Time Opened
aggregated_trades_df.sort_values(by='Time Opened', ascending=True, inplace=True)

# Reorder columns to match the desired order
aggregated_trades_df = aggregated_trades_df[
    ['Symbol', 'Description', 'Direction', 'Time Opened', 'Time Closed',
     'Holding Period (minutes)', 'Quantity', 'Average Buy Price',
     'Average Sell Price', 'Profit/Loss per Unit', 'Profit/Loss']
]

# Save the aggregated output to a CSV file
aggregated_trades_df.to_csv(output_file_path, index=False)

print(f"\nAggregated Trades saved to {output_file_path}")


Available files:
1: 2024-12-14-AccountStatement.csv
2: 2025-01-02-AccountStatement.csv
3: 2025-01-03-AccountStatement.csv
4: 2025-01-06-AccountStatement.csv


Enter the number corresponding to the file you want to process:  4


Selected file: C:\Users\bguru\Documents\Trading\Trade Analysis\TOS Statement Download\2025-01-06-AccountStatement.csv

Aggregated Trades saved to C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog\2025-01-06-Aggregated_Trades.csv


In [ ]:
# VERTICAL COMBO AGGREGATED TRADES
import csv
import os
import pandas as pd
from itertools import chain
from dateutil.parser import parse

# Directory to load files from
directory_path = r'C:\Users\bguru\Documents\Trading\Trade Analysis\TOS Statement Download'
output_directory = r'C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog'  # Directory for saving outputs

# List files in the directory
files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]
if not files:
    print(f"No files found in directory: {directory_path}")
    exit()

print("Available files:")
for idx, file in enumerate(files):
    print(f"{idx + 1}: {file}")

# Prompt the user to select a file
while True:
    try:
        file_choice = int(input("Enter the number corresponding to the file you want to process: "))
        if 1 <= file_choice <= len(files):
            file_path = os.path.join(directory_path, files[file_choice - 1])
            file_name = files[file_choice - 1]
            print(f"Selected file: {file_path}")
            break
        else:
            print("Invalid choice. Please select a valid file number.")
    except ValueError:
        print("Invalid input. Please enter a number.")

# Extract the date from the input file name (assuming the format is `YYYY-MM-DD-*`)
try:
    date_part = file_name.split('-')[0] + '-' + file_name.split('-')[1] + '-' + file_name.split('-')[2]
except IndexError:
    raise ValueError("The file name format does not contain a valid date.")

# Create output file paths with the date in the name
output_file_path = os.path.join(output_directory, f"{date_part}-Vertical_Combo_Aggregated_Trades.csv")

# Data storage
accountTradeHistory = []

# Read CSV file
with open(file_path) as statement:
    statement = csv.reader(statement, delimiter='\n')
    count, breakCount = 0, 0

    for line in statement:
        count += 1
        if count < 5:
            continue
        if len(line) == 0:
            breakCount += 1
        if breakCount == 6 and len(line) != 0:
            accountTradeHistory.append(line)

# Process CSV data into a pandas DataFrame
def orderSplitter(subsection: list) -> pd.DataFrame:
    subsection.pop(0)
    for i, j in enumerate(subsection):
        j[0] = j[0].split(',')

    return pd.DataFrame(
        list(chain.from_iterable(subsection[1:])),
        columns=list(chain.from_iterable(subsection[0]))
    ).iloc[:, 1:].iloc[::-1].reset_index().drop(['index'], axis=1)

orders = orderSplitter(accountTradeHistory)

# Filter only "VERTICAL" and "COMBO" spreads
filtered_orders = orders[
    orders['Spread'].isin(['VERTICAL', 'COMBO'])
].reset_index(drop=True)

# Group by symbols
def groupBySymbol(filtered_df):
    grouped = {}
    for symbol in set(filtered_df['Symbol']):
        grouped[symbol] = filtered_df[filtered_df['Symbol'] == symbol]
    return grouped

vertical_combo_orders_by_symbol = groupBySymbol(filtered_orders)

# Function to parse Description
def parseDescription(row):
    return f"{row['Spread']} {row['Exp']} {row['Strike']} {row['Type']}"

# Analyze trades
def processMatchedTrades(symbol, positions):
    openTrades = []
    aggregatedTrades = []

    for _, row in positions.iterrows():
        qty = int(row['Qty'].replace('+', '').replace('-', ''))
        price = float(row['Net Price'])
        timestamp = row['Exec Time']  # Use the correct column for timestamp
        description = parseDescription(row)

        # Updated Direction Logic
        if row['Pos Effect'] == 'TO OPEN' and row['Side'] == 'BUY':
            direction = 'Long'
        elif row['Pos Effect'] == 'TO OPEN' and row['Side'] == 'SELL':
            direction = 'Short'
        else:
            direction = None  # Direction not applicable for TO CLOSE

        if row['Pos Effect'] == 'TO OPEN':
            openTrades.append({
                'Symbol': symbol,
                'Description': description,
                'Direction': direction,
                'Time Opened': timestamp,
                'Time Closed': '',
                'Holding Period (minutes)': '',
                'Quantity': qty,
                'Average Buy Price': price,
                'Average Sell Price': '',
                'Profit/Loss per Unit': '',
                'Profit/Loss': ''
            })
        elif row['Pos Effect'] == 'TO CLOSE':
            sellQty = qty
            sellPrice = price
            sellTimestamp = timestamp

            weightedSellPrice = 0
            totalMatchedQty = 0
            weightedBuyPrice = 0
            buyTimestamp = None

            while sellQty > 0 and openTrades:
                buyTrade = openTrades.pop(0)
                matchQty = min(sellQty, buyTrade['Quantity'])
                sellQty -= matchQty
                buyTrade['Quantity'] -= matchQty

                weightedSellPrice += matchQty * sellPrice
                weightedBuyPrice += matchQty * buyTrade['Average Buy Price']
                totalMatchedQty += matchQty

                if buyTrade['Quantity'] > 0:
                    openTrades.insert(0, buyTrade)

                buyTimestamp = buyTrade['Time Opened']

            if totalMatchedQty > 0:
                avgSellPrice = weightedSellPrice / totalMatchedQty
                avgBuyPrice = weightedBuyPrice / totalMatchedQty
                profitLoss = (avgSellPrice - avgBuyPrice) * totalMatchedQty
                holdingPeriod = (parse(sellTimestamp) - parse(buyTimestamp)).total_seconds() / 60.0

                aggregatedTrades.append([
                    symbol, description, buyTrade['Direction'],
                    buyTimestamp, sellTimestamp, holdingPeriod, totalMatchedQty,
                    avgBuyPrice, avgSellPrice, profitLoss / totalMatchedQty, profitLoss * 100
                ])

    return aggregatedTrades

# Process and aggregate trades
trades = []

# Process vertical and combo trades
for symbol, positions in vertical_combo_orders_by_symbol.items():
    vertical_combo_aggregated = processMatchedTrades(symbol, positions)
    trades.extend(vertical_combo_aggregated)

# Save aggregated trades to a CSV file
aggregated_trades_df = pd.DataFrame(trades, columns=[
    'Symbol', 'Description', 'Direction', 'Time Opened', 'Time Closed',
    'Holding Period (minutes)', 'Quantity', 'Average Buy Price',
    'Average Sell Price', 'Profit/Loss per Unit', 'Profit/Loss'
])

# Combine rows with the same Symbol, Description, Direction, Time Opened, and Time Closed
# Round numerical columns to avoid precision issues
aggregated_trades_df['Holding Period (minutes)'] = pd.to_numeric(aggregated_trades_df['Holding Period (minutes)'], errors='coerce').round(2)
aggregated_trades_df['Average Buy Price'] = pd.to_numeric(aggregated_trades_df['Average Buy Price'], errors='coerce').round(2)
aggregated_trades_df['Average Sell Price'] = pd.to_numeric(aggregated_trades_df['Average Sell Price'], errors='coerce').round(2)
aggregated_trades_df['Profit/Loss per Unit'] = pd.to_numeric(aggregated_trades_df['Profit/Loss per Unit'], errors='coerce').round(2)
aggregated_trades_df['Profit/Loss'] = pd.to_numeric(aggregated_trades_df['Profit/Loss'], errors='coerce').round(2)

# Combine rows with the same Symbol, Description, Direction, Time Opened, and Time Closed
# Convert numeric columns and ignore minor differences by rounding
numeric_columns = [
    'Holding Period (minutes)', 'Average Buy Price',
    'Average Sell Price', 'Profit/Loss per Unit', 'Profit/Loss'
]

for col in numeric_columns:
    aggregated_trades_df[col] = pd.to_numeric(aggregated_trades_df[col], errors='coerce').round(2)

# Group by the specified columns
aggregated_trades_df = aggregated_trades_df.groupby(
    ['Symbol', 'Description', 'Direction', 'Time Opened', 'Time Closed', 'Average Buy Price'],
    as_index=False
).agg({
    'Holding Period (minutes)': 'sum',  # Sum holding periods to merge
    'Quantity': 'sum',  # Sum quantities
    'Average Sell Price': 'mean',  # Average sell price across rows
    'Profit/Loss per Unit': 'mean',  # Average profit/loss per unit
    'Profit/Loss': 'sum'  # Sum profit/loss
})

# Specify the date-time format
date_format = '%m/%d/%y %H:%M:%S'

# Convert and sort before saving
try:
    aggregated_trades_df['Time Opened'] = pd.to_datetime(
        aggregated_trades_df['Time Opened'], format=date_format, errors='coerce'
    )
    aggregated_trades_df['Time Closed'] = pd.to_datetime(
        aggregated_trades_df['Time Closed'], format=date_format, errors='coerce'
    )
except Exception as e:
    print(f"Error during date-time conversion: {e}")

# Sort by 'Time Opened'
aggregated_trades_df.sort_values(by='Time Opened', ascending=True, inplace=True)

# Save the output
aggregated_trades_df.to_csv(output_file_path, index=False)
print(f"\nAggregated Trades for VERTICAL and COMBO saved to {output_file_path}")


Available files:
1: 2024-12-14-AccountStatement.csv
2: 2025-01-02-AccountStatement.csv
3: 2025-01-03-AccountStatement.csv
4: 2025-01-06-AccountStatement.csv


Enter the number corresponding to the file you want to process:  4


Selected file: C:\Users\bguru\Documents\Trading\Trade Analysis\TOS Statement Download\2025-01-06-AccountStatement.csv

Aggregated Trades for VERTICAL and COMBO saved to C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog\2025-01-06-Vertical_Combo_Aggregated_Trades.csv


In [ ]:
# OPEN, CLOSED, AND OPEN+CLOSED AGGREGATED

import csv
import os
import pandas as pd
from dateutil.parser import parse

# Define directories
base_directory = r'C:\Users\bguru\Documents\Trading\Trade Analysis'
tos_directory = os.path.join(base_directory, 'TOS Statement Download')
output_directory = os.path.join(base_directory, 'Tradelog', 'Open_Close_Trades')
aggregated_output_directory = os.path.join(base_directory, 'Tradelog')

# Ensure output directories exist
os.makedirs(output_directory, exist_ok=True)
os.makedirs(aggregated_output_directory, exist_ok=True)

# List files in the TOS Statement directory
files = [f for f in os.listdir(tos_directory) if os.path.isfile(os.path.join(tos_directory, f))]
if not files:
    print(f"No files found in directory: {tos_directory}")
    exit()

print("Available Account Statement files:")
for idx, file in enumerate(files):
    print(f"{idx + 1}: {file}")


def parse_account_statement(file_path):
    """Parse the account statement file."""
    cashBalance = []
    accountTradeHistory = []
    with open(file_path) as statement:
        statement = csv.reader(statement, delimiter='\n')
        count, breakCount = 0, 0
        for line in statement:
            count += 1
            if count < 5:
                continue
            if len(line) == 0:
                breakCount += 1
            if breakCount < 1:
                cashBalance.append(line)
            if breakCount == 6 and len(line) != 0:
                accountTradeHistory.append(line)
    return accountTradeHistory


def orderSplitter(subsection: list) -> pd.DataFrame:
    """Split the order data from the parsed account statement."""
    subsection.pop(0)
    for i, j in enumerate(subsection):
        j[0] = j[0].split(',')
    return pd.DataFrame(
        list(chain.from_iterable(subsection[1:])),
        columns=list(chain.from_iterable(subsection[0]))
    ).iloc[:, 1:].iloc[::-1].reset_index().drop(['index'], axis=1)


def create_open_trades():
    """Create the Open Trades file from the selected Account Statement."""
    while True:
        try:
            open_file_choice = int(input("Choose file for Open Trades (enter the number corresponding to the file): "))
            if 1 <= open_file_choice <= len(files):
                open_file_path = os.path.join(tos_directory, files[open_file_choice - 1])
                open_file_name = files[open_file_choice - 1]
                print(f"Selected file for Open Trades: {open_file_path}")
                break
            else:
                print("Invalid choice. Please select a valid file number.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    try:
        open_date = open_file_name.split('-')[0] + '-' + open_file_name.split('-')[1] + '-' + open_file_name.split('-')[2]
    except IndexError:
        raise ValueError("The file name format does not contain a valid date.")

    open_trades_output_path = os.path.join(output_directory, f"{open_date} Open Trades.csv")
    account_trade_history = parse_account_statement(open_file_path)
    account_orders = orderSplitter(account_trade_history)

    filtered_orders = account_orders[
        ~account_orders['Spread'].isin(['VERTICAL', 'COMBO', 'COVERED', ''])
    ].reset_index(drop=True)

    open_trades = filtered_orders[
        (filtered_orders['Spread'].isin(['STOCK', 'SINGLE'])) & (filtered_orders['Pos Effect'] == 'TO OPEN')
    ]

    unmatched_open_trades = filterTrueOpenTrades(open_trades, filtered_orders)
    unmatched_open_trades.to_csv(open_trades_output_path, index=False)
    print(f"Open Trades saved to {open_trades_output_path}")
    return open_trades_output_path, open_date


def filterTrueOpenTrades(open_trades, all_trades):
    """Identify open trades that have no corresponding TO CLOSE."""
    open_trades = open_trades.copy()
    all_trades = all_trades.copy()

    open_trades['Quantity'] = open_trades['Qty'].astype(int)
    all_trades['Quantity'] = all_trades['Qty'].astype(int).abs()

    remaining_open_trades = open_trades.copy()

    for _, close_trade in all_trades[all_trades['Pos Effect'] == 'TO CLOSE'].iterrows():
        symbol = close_trade['Symbol']
        qty_to_close = close_trade['Quantity']

        matching_open_trades = remaining_open_trades[
            (remaining_open_trades['Symbol'] == symbol) & (remaining_open_trades['Quantity'] > 0)
        ]

        for idx, open_trade in matching_open_trades.iterrows():
            if qty_to_close <= 0:
                break

            match_qty = min(qty_to_close, open_trade['Quantity'])
            qty_to_close -= match_qty
            remaining_open_trades.loc[idx, 'Quantity'] -= match_qty

    return remaining_open_trades[remaining_open_trades['Quantity'] > 0]


def create_closed_trades(open_trades_path, open_date):
    """Create the Closed Trades file from the unmatched Open Trades."""
    print("\nCreating Closed Trades...")
    while True:
        try:
            closed_file_choice = int(input("Choose file for Account Statement to create Closed Trades: "))
            if 1 <= closed_file_choice <= len(files):
                closed_file_path = os.path.join(tos_directory, files[closed_file_choice - 1])
                closed_file_name = files[closed_file_choice - 1]
                print(f"Selected Account Statement file: {closed_file_path}")
                break
            else:
                print("Invalid choice. Please select a valid file number.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    try:
        closed_date = closed_file_name.split('-')[0] + '-' + closed_file_name.split('-')[1] + '-' + closed_file_name.split('-')[2]
    except IndexError:
        raise ValueError("The file name format does not contain a valid date.")

    closed_trades_output_path = os.path.join(output_directory, f"{closed_date} Closed Trades.csv")
    open_trades = pd.read_csv(open_trades_path)

    account_trade_history = parse_account_statement(closed_file_path)
    account_orders = orderSplitter(account_trade_history)

    # Filter and handle non-numeric values in 'Qty'
    closed_trades = account_orders[
        (account_orders['Spread'].isin(['STOCK', 'SINGLE'])) & (account_orders['Pos Effect'] == 'TO CLOSE')
    ].reset_index(drop=True)

    # Convert 'Qty' to numeric, coercing errors to NaN, then drop non-numeric rows
    closed_trades['Qty'] = pd.to_numeric(closed_trades['Qty'], errors='coerce').abs()
    closed_trades = closed_trades.dropna(subset=['Qty'])  # Drop rows where 'Qty' could not be converted
    closed_trades['Qty'] = closed_trades['Qty'].astype(int)

    matched_closed_trades = matchClosedTradesWithOpenTrades(open_trades, closed_trades)

    if not matched_closed_trades.empty:
        matched_closed_trades.to_csv(closed_trades_output_path, index=False)
        print(f"Closed Trades saved to {closed_trades_output_path}")
        return closed_trades_output_path, closed_date
    else:
        print("No matched closed trades found. File not created.")
        return None, None


def matchClosedTradesWithOpenTrades(open_trades, closed_trades):
    matched_closed_trades = []
    open_trades_copy = open_trades.copy()

    for _, open_trade in open_trades_copy.iterrows():
        symbol = open_trade['Symbol']
        qty_to_match = open_trade['Quantity']

        matching_closed_trades = closed_trades[
            (closed_trades['Symbol'] == symbol) & (closed_trades['Qty'] > 0)
        ]

        for idx, closed_trade in matching_closed_trades.iterrows():
            if qty_to_match <= 0:
                break

            match_qty = min(qty_to_match, closed_trade['Qty'])
            qty_to_match -= match_qty

            if match_qty > 0:
                matched_trade = closed_trade.copy()
                matched_trade['Matched Quantity'] = match_qty
                matched_closed_trades.append(matched_trade)

            closed_trades.at[idx, 'Qty'] -= match_qty

    return pd.DataFrame(matched_closed_trades)


def aggregate_trades(open_date, closed_date, open_trades_path, closed_trades_path):
    """Aggregate Open and Closed Trades."""
    print("\nAggregating Trades...")
    aggregated_trades_output_path = os.path.join(aggregated_output_directory, f"{open_date}_to_{closed_date}_Aggregated_Trades.csv")

    open_trades = pd.read_csv(open_trades_path)
    closed_trades = pd.read_csv(closed_trades_path)

    # Ensure proper data types
    open_trades['Quantity'] = open_trades['Quantity'].astype(int)
    closed_trades['Qty'] = closed_trades['Qty'].astype(int).abs()

    def processMatchedTrades(open_trades, closed_trades):
        """Match and aggregate open and closed trades."""
        aggregated_trades = []
        remaining_open_trades = open_trades.copy()

        for _, closed in closed_trades.iterrows():
            symbol = closed['Symbol']
            qty_to_close = closed['Qty']
            sell_price = float(closed['Price'])
            close_time = closed['Exec Time']

            matching_open_trades = remaining_open_trades[
                (remaining_open_trades['Symbol'] == symbol) & (remaining_open_trades['Quantity'] > 0)
            ]

            for idx, open_trade in matching_open_trades.iterrows():
                if qty_to_close <= 0:
                    break

                match_qty = min(qty_to_close, open_trade['Quantity'])
                qty_to_close -= match_qty
                remaining_open_trades.loc[idx, 'Quantity'] -= match_qty

                avg_buy_price = float(open_trade['Price'])
                open_time = open_trade['Exec Time']

                profit_loss = (sell_price - avg_buy_price) * match_qty
                holding_period = (parse(close_time) - parse(open_time)).total_seconds() / 60.0

                description = 'Stock' if open_trade['Spread'] == 'STOCK' else f"{open_trade['Spread']} {open_trade['Exp']} {open_trade['Strike']} {open_trade['Type']}"

                aggregated_trades.append({
                    'Symbol': symbol,
                    'Description': description,
                    'Direction': 'Long',
                    'Time Opened': open_time,
                    'Time Closed': close_time,
                    'Holding Period (minutes)': holding_period,
                    'Quantity': match_qty,
                    'Average Buy Price': avg_buy_price,
                    'Average Sell Price': sell_price,
                    'Profit/Loss per Unit': profit_loss / match_qty,
                    'Profit/Loss': profit_loss
                })

        return pd.DataFrame(aggregated_trades)

    aggregated_trades = processMatchedTrades(open_trades, closed_trades)

    # Convert 'Time Opened' and 'Time Closed' to datetime explicitly
    datetime_format = "%m/%d/%y %H:%M:%S"
    aggregated_trades['Time Opened'] = pd.to_datetime(aggregated_trades['Time Opened'], format=datetime_format, errors='coerce')
    aggregated_trades['Time Closed'] = pd.to_datetime(aggregated_trades['Time Closed'], format=datetime_format, errors='coerce')

    # Group by Symbol, Description, and Time Opened while aggregating other values
    if not aggregated_trades.empty:
        numeric_columns = [
            'Holding Period (minutes)', 'Quantity', 'Average Buy Price',
            'Average Sell Price', 'Profit/Loss per Unit', 'Profit/Loss'
        ]

        for col in numeric_columns:
            aggregated_trades[col] = pd.to_numeric(aggregated_trades[col], errors='coerce')

        aggregated_trades = aggregated_trades.groupby(
            ['Symbol', 'Description', 'Time Opened'],
            as_index=False
        ).agg({
            'Direction': 'first',  # Keep the first Direction value
            'Time Closed': 'max',  # Take the latest Time Closed value
            'Holding Period (minutes)': 'sum',  # Sum Holding Period
            'Quantity': 'sum',  # Sum Quantity
            'Average Buy Price': 'mean',  # Average Buy Price
            'Average Sell Price': 'mean',  # Average Sell Price
            'Profit/Loss per Unit': 'mean',  # Average Profit/Loss per Unit
            'Profit/Loss': 'sum'  # Sum Profit/Loss
        })

        # Apply conversion only for SINGLE trades
        aggregated_trades['Profit/Loss'] = aggregated_trades['Profit/Loss'].astype(float).round(2)
        single_mask = aggregated_trades['Description'].str.startswith('SINGLE', na=False)
        aggregated_trades.loc[single_mask, 'Profit/Loss'] = (
            pd.to_numeric(aggregated_trades.loc[single_mask, 'Profit/Loss'], errors='coerce').fillna(0) * 100
        ).round(2)

        # Convert 'Profit/Loss' back to string for saving
        aggregated_trades['Profit/Loss'] = aggregated_trades['Profit/Loss'].astype(str)

        # Sort the aggregated trades by Time Opened
        aggregated_trades.sort_values(by='Time Opened', ascending=True, inplace=True)

        # Reorder columns to match the desired order
        aggregated_trades = aggregated_trades[
            ['Symbol', 'Description', 'Direction', 'Time Opened', 'Time Closed',
             'Holding Period (minutes)', 'Quantity', 'Average Buy Price',
             'Average Sell Price', 'Profit/Loss per Unit', 'Profit/Loss']
        ]

        # Save to file
        aggregated_trades.to_csv(aggregated_trades_output_path, index=False)
        print(f"Aggregated Trades saved to {aggregated_trades_output_path}")
    else:
        print(f"No aggregated trades found. Empty file saved to {aggregated_trades_output_path}")
        pd.DataFrame().to_csv(aggregated_trades_output_path, index=False, header=True)


# Run the process
open_trades_path, open_date = create_open_trades()
if open_trades_path:
    closed_trades_path, closed_date = create_closed_trades(open_trades_path, open_date)
    if closed_trades_path:
        aggregate_trades(open_date, closed_date, open_trades_path, closed_trades_path)


Available Account Statement files:
1: 2024-12-14-AccountStatement.csv
2: 2025-01-02-AccountStatement.csv
3: 2025-01-03-AccountStatement.csv


Choose file for Open Trades (enter the number corresponding to the file):  1


Selected file for Open Trades: C:\Users\bguru\Documents\Trading\Trade Analysis\TOS Statement Download\2024-12-14-AccountStatement.csv
Open Trades saved to C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog\Open_Close_Trades\2024-12-14 Open Trades.csv

Creating Closed Trades...


Choose file for Account Statement to create Closed Trades:  2


Selected Account Statement file: C:\Users\bguru\Documents\Trading\Trade Analysis\TOS Statement Download\2025-01-02-AccountStatement.csv
Closed Trades saved to C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog\Open_Close_Trades\2025-01-02 Closed Trades.csv

Aggregating Trades...
Aggregated Trades saved to C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog\2024-12-14_to_2025-01-02_Aggregated_Trades.csv


In [ ]:
# VERTICAL/COMBO OPEN, CLOSED, AND OPEN+CLOSED AGGREGATED

import csv
import os
import pandas as pd
from dateutil.parser import parse

# Define directories
base_directory = r'C:\Users\bguru\Documents\Trading\Trade Analysis'
tos_directory = os.path.join(base_directory, 'TOS Statement Download')
vertical_combo_output_directory = os.path.join(base_directory, 'Tradelog', 'Open_Close_Trades')
aggregated_output_directory = os.path.join(base_directory, 'Tradelog')

# Ensure output directories exist
os.makedirs(vertical_combo_output_directory, exist_ok=True)
os.makedirs(aggregated_output_directory, exist_ok=True)

# List files in the TOS Statement directory
files = [f for f in os.listdir(tos_directory) if os.path.isfile(os.path.join(tos_directory, f))]
if not files:
    print(f"No files found in directory: {tos_directory}")
    exit()

print("Available Account Statement files:")
for idx, file in enumerate(files):
    print(f"{idx + 1}: {file}")


def parse_account_statement(file_path):
    """Parse the account statement file."""
    cashBalance = []
    accountTradeHistory = []
    with open(file_path) as statement:
        statement = csv.reader(statement, delimiter='\n')
        count, breakCount = 0, 0
        for line in statement:
            count += 1
            if count < 5:
                continue
            if len(line) == 0:
                breakCount += 1
            if breakCount < 1:
                cashBalance.append(line)
            if breakCount == 6 and len(line) != 0:
                accountTradeHistory.append(line)
    return accountTradeHistory


def orderSplitter(subsection: list) -> pd.DataFrame:
    """Split the order data from the parsed account statement."""
    subsection.pop(0)
    for i, j in enumerate(subsection):
        j[0] = j[0].split(',')
    return pd.DataFrame(
        list(chain.from_iterable(subsection[1:])),
        columns=list(chain.from_iterable(subsection[0]))
    ).iloc[:, 1:].iloc[::-1].reset_index().drop(['index'], axis=1)


def parseDescription(row):
    """Parse the description for vertical/combo trades."""
    return f"{row['Spread']} {row['Exp']} {row['Strike']} {row['Type']}"


def create_open_trades_vertical_combo():
    """Create the Vertical/Combo Open Trades file."""
    while True:
        try:
            open_file_choice = int(input("Choose file for Vertical/Combo Open Trades (enter the number corresponding to the file): "))
            if 1 <= open_file_choice <= len(files):
                open_file_path = os.path.join(tos_directory, files[open_file_choice - 1])
                open_file_name = files[open_file_choice - 1]
                print(f"Selected file for Vertical/Combo Open Trades: {open_file_path}")
                break
            else:
                print("Invalid choice. Please select a valid file number.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    try:
        open_date = open_file_name.split('-')[0] + '-' + open_file_name.split('-')[1] + '-' + open_file_name.split('-')[2]
    except IndexError:
        raise ValueError("The file name format does not contain a valid date.")

    open_trades_output_path = os.path.join(vertical_combo_output_directory, f"{open_date} Vertical_Combo_Open_Trades.csv")
    account_trade_history = parse_account_statement(open_file_path)
    account_orders = orderSplitter(account_trade_history)

    vertical_combo_open_trades = account_orders[
        (account_orders['Spread'].isin(['VERTICAL', 'COMBO'])) & (account_orders['Pos Effect'] == 'TO OPEN')
    ].copy()

    # Add parsed description
    vertical_combo_open_trades['Description'] = vertical_combo_open_trades.apply(parseDescription, axis=1)

    vertical_combo_open_trades.to_csv(open_trades_output_path, index=False)
    print(f"Vertical/Combo Open Trades saved to {open_trades_output_path}")
    return open_trades_output_path, open_date


def create_closed_trades_vertical_combo(open_trades_path, open_date):
    """Create the Vertical/Combo Closed Trades file."""
    print("\nCreating Vertical/Combo Closed Trades...")
    while True:
        try:
            closed_file_choice = int(input("Choose file for Account Statement to create Closed Trades: "))
            if 1 <= closed_file_choice <= len(files):
                closed_file_path = os.path.join(tos_directory, files[closed_file_choice - 1])
                closed_file_name = files[closed_file_choice - 1]
                print(f"Selected Account Statement file: {closed_file_path}")
                break
            else:
                print("Invalid choice. Please select a valid file number.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    try:
        closed_date = closed_file_name.split('-')[0] + '-' + closed_file_name.split('-')[1] + '-' + closed_file_name.split('-')[2]
    except IndexError:
        raise ValueError("The file name format does not contain a valid date.")

    closed_trades_output_path = os.path.join(vertical_combo_output_directory, f"{closed_date} Vertical_Combo_Closed_Trades.csv")
    account_trade_history = parse_account_statement(closed_file_path)
    account_orders = orderSplitter(account_trade_history)

    vertical_combo_closed_trades = account_orders[
        (account_orders['Spread'].isin(['VERTICAL', 'COMBO'])) & (account_orders['Pos Effect'] == 'TO CLOSE')
    ].copy()

    # Add parsed description
    vertical_combo_closed_trades['Description'] = vertical_combo_closed_trades.apply(parseDescription, axis=1)

    vertical_combo_closed_trades.to_csv(closed_trades_output_path, index=False)
    print(f"Vertical/Combo Closed Trades saved to {closed_trades_output_path}")
    return closed_trades_output_path, closed_date


def aggregate_vertical_combo_trades(open_date, closed_date, open_trades_path, closed_trades_path):
    """Aggregate Vertical/Combo Open and Closed Trades."""
    print("\nAggregating Vertical/Combo Trades...")
    aggregated_trades_output_path = os.path.join(aggregated_output_directory, f"{open_date}_to_{closed_date}_Vertical_Combo_Aggregated_Trades.csv")

    open_trades = pd.read_csv(open_trades_path)
    closed_trades = pd.read_csv(closed_trades_path)

    # Ensure 'Description' exists in both dataframes
    if 'Description' not in open_trades.columns:
        open_trades['Description'] = open_trades.apply(parseDescription, axis=1)
    if 'Description' not in closed_trades.columns:
        closed_trades['Description'] = closed_trades.apply(parseDescription, axis=1)

    def processMatchedTrades(open_trades, closed_trades):
        aggregated_trades = []
        remaining_open_trades = open_trades.copy()

        for _, closed in closed_trades.iterrows():
            symbol = closed['Symbol']
            qty_to_close = int(str(closed['Qty']).replace('+', '').replace('-', '').strip())
            sell_price = float(closed['Net Price'])
            close_time = closed['Exec Time']

            matching_open_trades = remaining_open_trades[
                (remaining_open_trades['Symbol'] == symbol) &
                (remaining_open_trades['Qty'].astype(str).str.replace('+', '').str.replace('-', '').astype(int) > 0)
            ]

            for idx, open_trade in matching_open_trades.iterrows():
                if qty_to_close <= 0:
                    break

                match_qty = min(qty_to_close, int(str(open_trade['Qty']).replace('+', '').replace('-', '').strip()))
                qty_to_close -= match_qty

                remaining_open_trades.loc[idx, 'Qty'] = (
                    int(str(open_trade['Qty']).replace('+', '').replace('-', '').strip()) - match_qty
                )

                avg_buy_price = float(open_trade['Net Price'])
                open_time = open_trade['Exec Time']

                profit_loss = (sell_price - avg_buy_price) * match_qty
                holding_period = (parse(close_time) - parse(open_time)).total_seconds() / 60.0

                direction = 'Long' if open_trade['Side'] == 'BUY' else 'Short'

                aggregated_trades.append({
                    'Symbol': symbol,
                    'Description': open_trade['Description'],
                    'Direction': direction,
                    'Time Opened': open_time,
                    'Time Closed': close_time,
                    'Holding Period (minutes)': holding_period,
                    'Quantity': match_qty,
                    'Average Buy Price': avg_buy_price,
                    'Average Sell Price': sell_price,
                    'Profit/Loss per Unit': profit_loss / match_qty,
                    'Profit/Loss': profit_loss * 100  # Convert to percentage
                })

        return pd.DataFrame(aggregated_trades)

    aggregated_trades = processMatchedTrades(open_trades, closed_trades)

    if not aggregated_trades.empty:
        # Convert numeric columns to ensure consistent grouping and calculations
        numeric_columns = [
            'Holding Period (minutes)', 'Quantity', 'Average Buy Price',
            'Average Sell Price', 'Profit/Loss per Unit', 'Profit/Loss'
        ]
        for col in numeric_columns:
            aggregated_trades[col] = pd.to_numeric(aggregated_trades[col], errors='coerce').round(2)

        # Group by key columns and sum/average relevant fields
        grouped_trades = aggregated_trades.groupby(
            ['Symbol', 'Description', 'Direction', 'Time Opened', 'Time Closed'],
            as_index=False
        ).agg({
            'Holding Period (minutes)': 'sum',
            'Quantity': 'sum',
            'Average Buy Price': 'first',
            'Average Sell Price': 'mean',
            'Profit/Loss per Unit': 'mean',
            'Profit/Loss': 'sum'
        })

        # Convert 'Time Opened' and 'Time Closed' to datetime for sorting
        date_format = '%m/%d/%y %H:%M:%S'
        grouped_trades['Time Opened'] = pd.to_datetime(grouped_trades['Time Opened'], format=date_format, errors='coerce')
        grouped_trades['Time Closed'] = pd.to_datetime(grouped_trades['Time Closed'], format=date_format, errors='coerce')

        # Sort by 'Time Opened'
        grouped_trades.sort_values(by='Time Opened', ascending=True, inplace=True)

        # Convert Profit/Loss back to string format for saving
        grouped_trades['Profit/Loss'] = grouped_trades['Profit/Loss'].round(2).astype(str)

        # Save the output
        grouped_trades.to_csv(aggregated_trades_output_path, index=False)
        print(f"Vertical/Combo Aggregated Trades saved to {aggregated_trades_output_path}")
    else:
        print(f"No aggregated trades found. Empty file saved to {aggregated_trades_output_path}")
        pd.DataFrame().to_csv(aggregated_trades_output_path, index=False, header=True)

# Run the process
open_trades_path, open_date = create_open_trades_vertical_combo()
if open_trades_path:
    closed_trades_path, closed_date = create_closed_trades_vertical_combo(open_trades_path, open_date)
    if closed_trades_path:
        aggregate_vertical_combo_trades(open_date, closed_date, open_trades_path, closed_trades_path)


Available Account Statement files:
1: 2024-12-14-AccountStatement.csv
2: 2025-01-02-AccountStatement.csv
3: 2025-01-03-AccountStatement.csv


Choose file for Vertical/Combo Open Trades (enter the number corresponding to the file):  1


Selected file for Vertical/Combo Open Trades: C:\Users\bguru\Documents\Trading\Trade Analysis\TOS Statement Download\2024-12-14-AccountStatement.csv
Vertical/Combo Open Trades saved to C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog\Open_Close_Trades\2024-12-14 Vertical_Combo_Open_Trades.csv

Creating Vertical/Combo Closed Trades...


Choose file for Account Statement to create Closed Trades:  2


Selected Account Statement file: C:\Users\bguru\Documents\Trading\Trade Analysis\TOS Statement Download\2025-01-02-AccountStatement.csv
Vertical/Combo Closed Trades saved to C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog\Open_Close_Trades\2025-01-02 Vertical_Combo_Closed_Trades.csv

Aggregating Vertical/Combo Trades...
No aggregated trades found. Empty file saved to C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog\2024-12-14_to_2025-01-02_Vertical_Combo_Aggregated_Trades.csv


In [ ]:
## MERGE NEW TRADELOG INFO TO ALL TRADES TRADELOG

import os
import pandas as pd

# Directory to load files from
directory_path = r'C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog'

# List files in the directory
files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]
if not files:
    print(f"No files found in directory: {directory_path}")
    exit()

print("Available files:")
for idx, file in enumerate(files):
    print(f"{idx + 1}: {file}")

# Prompt the user to select the existing All Trades Tradelog
while True:
    try:
        file_choice1 = int(input("Choose the All Trades Tradelog file (enter the number corresponding to the file): "))
        if 1 <= file_choice1 <= len(files):
            all_trades_file = os.path.join(directory_path, files[file_choice1 - 1])
            print(f"Selected All Trades Tradelog file: {all_trades_file}")
            break
        else:
            print("Invalid choice. Please select a valid file number.")
    except ValueError:
        print("Invalid input. Please enter a number.")

# Read the All Trades Tradelog file
try:
    all_trades = pd.read_csv(all_trades_file, dtype=str)  # Ensure all columns are read as strings
except Exception as e:
    print(f"Error reading the All Trades Tradelog file: {e}")
    exit()

# Prompt the user to add new files until they type "done"
while True:
    print("\nAvailable files:")
    for idx, file in enumerate(files):
        print(f"{idx + 1}: {file}")
    print("Type 'done' to finish adding files.")

    file_choice = input("Choose the new aggregated trade file to merge (enter the number corresponding to the file): ")

    if file_choice.lower() == "done":
        print("Finished adding files.")
        break

    try:
        file_choice = int(file_choice)
        if 1 <= file_choice <= len(files):
            new_aggregated_file = os.path.join(directory_path, files[file_choice - 1])
            print(f"Selected aggregated trade file: {new_aggregated_file}")
        else:
            print("Invalid choice. Please select a valid file number.")
            continue
    except ValueError:
        print("Invalid input. Please enter a number or 'done' to finish.")
        continue

    # Read the new aggregated trade file
    try:
        new_aggregated_trades = pd.read_csv(new_aggregated_file, dtype=str)  # Ensure all columns are read as strings
    except Exception as e:
        print(f"Error reading the new aggregated trade file: {e}")
        continue

    # Merge the new aggregated trade file into the All Trades Tradelog
    all_trades = pd.concat([all_trades, new_aggregated_trades], ignore_index=True)

# Ensure "Time Opened" is treated as a string and sort it
all_trades = all_trades.sort_values(by='Time Opened', ascending=True, key=lambda x: x.astype(str))

# Save the updated All Trades Tradelog
output_file_path = os.path.join(directory_path, "All Trades Tradelog.csv")
all_trades.to_csv(output_file_path, index=False)

print(f"All Trades Tradelog successfully updated and saved to {output_file_path}")


Available files:
1: 2024-12-14-Aggregated_Trades - Copy.csv
2: 2024-12-14-Aggregated_Trades.csv
3: 2024-12-14-Vertical_Combo_Aggregated_Trades.csv
4: 2024-12-14_to_2025-01-02_Aggregated_Trades.csv
5: 2024-12-14_to_2025-01-02_Vertical_Combo_Aggregated_Trades.csv
6: 2025-01-02-Aggregated_Trades.csv
7: 2025-01-02-Vertical_Combo_Aggregated_Trades.csv
8: 2025-01-03-Aggregated_Trades.csv
9: 2025-01-03-Vertical_Combo_Aggregated_Trades.csv
10: All Trades Tradelog - Copy.csv
11: All Trades Tradelog.csv


Choose the All Trades Tradelog file (enter the number corresponding to the file):  11


Selected All Trades Tradelog file: C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog\All Trades Tradelog.csv

Available files:
1: 2024-12-14-Aggregated_Trades - Copy.csv
2: 2024-12-14-Aggregated_Trades.csv
3: 2024-12-14-Vertical_Combo_Aggregated_Trades.csv
4: 2024-12-14_to_2025-01-02_Aggregated_Trades.csv
5: 2024-12-14_to_2025-01-02_Vertical_Combo_Aggregated_Trades.csv
6: 2025-01-02-Aggregated_Trades.csv
7: 2025-01-02-Vertical_Combo_Aggregated_Trades.csv
8: 2025-01-03-Aggregated_Trades.csv
9: 2025-01-03-Vertical_Combo_Aggregated_Trades.csv
10: All Trades Tradelog - Copy.csv
11: All Trades Tradelog.csv
Type 'done' to finish adding files.


Choose the new aggregated trade file to merge (enter the number corresponding to the file):  4


Selected aggregated trade file: C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog\2024-12-14_to_2025-01-02_Aggregated_Trades.csv

Available files:
1: 2024-12-14-Aggregated_Trades - Copy.csv
2: 2024-12-14-Aggregated_Trades.csv
3: 2024-12-14-Vertical_Combo_Aggregated_Trades.csv
4: 2024-12-14_to_2025-01-02_Aggregated_Trades.csv
5: 2024-12-14_to_2025-01-02_Vertical_Combo_Aggregated_Trades.csv
6: 2025-01-02-Aggregated_Trades.csv
7: 2025-01-02-Vertical_Combo_Aggregated_Trades.csv
8: 2025-01-03-Aggregated_Trades.csv
9: 2025-01-03-Vertical_Combo_Aggregated_Trades.csv
10: All Trades Tradelog - Copy.csv
11: All Trades Tradelog.csv
Type 'done' to finish adding files.


...

Selected aggregated trade file: C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog\2024-12-14_to_2025-01-02_Vertical_Combo_Aggregated_Trades.csv
Error reading the new aggregated trade file: No columns to parse from file

Available files:
1: 2024-12-14-Aggregated_Trades - Copy.csv
2: 2024-12-14-Aggregated_Trades.csv
3: 2024-12-14-Vertical_Combo_Aggregated_Trades.csv
4: 2024-12-14_to_2025-01-02_Aggregated_Trades.csv
5: 2024-12-14_to_2025-01-02_Vertical_Combo_Aggregated_Trades.csv
6: 2025-01-02-Aggregated_Trades.csv
7: 2025-01-02-Vertical_Combo_Aggregated_Trades.csv
8: 2025-01-03-Aggregated_Trades.csv
9: 2025-01-03-Vertical_Combo_Aggregated_Trades.csv
10: All Trades Tradelog - Copy.csv
11: All Trades Tradelog.csv
Type 'done' to finish adding files.


Choose the new aggregated trade file to merge (enter the number corresponding to the file):  7


Selected aggregated trade file: C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog\2025-01-02-Vertical_Combo_Aggregated_Trades.csv

Available files:
1: 2024-12-14-Aggregated_Trades - Copy.csv
2: 2024-12-14-Aggregated_Trades.csv
3: 2024-12-14-Vertical_Combo_Aggregated_Trades.csv
4: 2024-12-14_to_2025-01-02_Aggregated_Trades.csv
5: 2024-12-14_to_2025-01-02_Vertical_Combo_Aggregated_Trades.csv
6: 2025-01-02-Aggregated_Trades.csv
7: 2025-01-02-Vertical_Combo_Aggregated_Trades.csv
8: 2025-01-03-Aggregated_Trades.csv
9: 2025-01-03-Vertical_Combo_Aggregated_Trades.csv
10: All Trades Tradelog - Copy.csv
11: All Trades Tradelog.csv
Type 'done' to finish adding files.


Choose the new aggregated trade file to merge (enter the number corresponding to the file):  done


Finished adding files.
All Trades Tradelog successfully updated and saved to C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog\All Trades Tradelog.csv


In [ ]:
import os
import pandas as pd

#####
## This is creating All Trades FIRST TIME AROUND when it doesn't exist.
#####

# Directory to load files from
directory_path = r'C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog'

# List files in the directory
files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]
if not files:
    print(f"No files found in directory: {directory_path}")
    exit()

print("Available files:")
for idx, file in enumerate(files):
    print(f"{idx + 1}: {file}")

# Prompt the user to select the first aggregated trade file
while True:
    try:
        file_choice1 = int(input("Choose the first aggregated trade file (enter the number corresponding to the file): "))
        if 1 <= file_choice1 <= len(files):
            file_path1 = os.path.join(directory_path, files[file_choice1 - 1])
            print(f"Selected file 1: {file_path1}")
            break
        else:
            print("Invalid choice. Please select a valid file number.")
    except ValueError:
        print("Invalid input. Please enter a number.")

# Prompt the user to select the second aggregated trade file
while True:
    try:
        file_choice2 = int(input("Choose the second aggregated trade file (enter the number corresponding to the file): "))
        if 1 <= file_choice2 <= len(files):
            file_path2 = os.path.join(directory_path, files[file_choice2 - 1])
            print(f"Selected file 2: {file_path2}")
            break
        else:
            print("Invalid choice. Please select a valid file number.")
    except ValueError:
        print("Invalid input. Please enter a number.")

# Read both files into pandas DataFrames
try:
    aggregated_trades1 = pd.read_csv(file_path1)
    aggregated_trades2 = pd.read_csv(file_path2)
except Exception as e:
    print(f"Error reading one of the files: {e}")
    exit()

# Combine the two files into one DataFrame
all_trades = pd.concat([aggregated_trades1, aggregated_trades2], ignore_index=True)

# Sort the combined DataFrame by the "Time Opened" column as a string
all_trades = all_trades.sort_values(by='Time Opened', ascending=True, key=lambda x: x.astype(str))

# Save the combined and sorted DataFrame to a CSV file
output_file_path = os.path.join(directory_path, "All Trades Tradelog.csv")
all_trades.to_csv(output_file_path, index=False)

print(f"\nAll Trades Tradelog saved to {output_file_path} (sorted by 'Time Opened')")


Available files:
1: 2024-12-14-Aggregated_Trades - Copy.csv
2: 2024-12-14-Aggregated_Trades.csv
3: 2024-12-14-Vertical_Combo_Aggregated_Trades.csv
4: 2024-12-14_to_2025-01-02_Aggregated_Trades.csv
5: 2024-12-14_to_2025-01-02_Vertical_Combo_Aggregated_Trades.csv
6: 2025-01-02-Aggregated_Trades.csv
7: 2025-01-02-Vertical_Combo_Aggregated_Trades.csv
8: 2025-01-03-Aggregated_Trades.csv
9: 2025-01-03-Vertical_Combo_Aggregated_Trades.csv
10: 2025-01-06-Aggregated_Trades.csv
11: 2025-01-06-Vertical_Combo_Aggregated_Trades.csv
12: All Trades Tradelog - Copy (2).csv
13: All Trades Tradelog.csv


Choose the first aggregated trade file (enter the number corresponding to the file):  10


Selected file 1: C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog\2025-01-06-Aggregated_Trades.csv


Choose the second aggregated trade file (enter the number corresponding to the file):  11


Selected file 2: C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog\2025-01-06-Vertical_Combo_Aggregated_Trades.csv

All Trades Tradelog saved to C:\Users\bguru\Documents\Trading\Trade Analysis\Tradelog\All Trades Tradelog.csv (sorted by 'Time Opened')
